# Duplication

Пример отчета по расчету пересечения аудитории интернет-ресурсов (Cross Web). 

## Описание задачи

Посчитаем пересечение аудитории интернет-ресурсов Avito и Ozon. 

Общие параметры:
- Период: Август 2022
- География: Россия 0+
- Население: 12+
- Типы пользования интернетом: ограничения нет, считаем по всем (Web Desktop, Web Mobile, App Mobile)

Статистики:
- Reach (reach)
- Reach% (reachPer)
- Average Daily Reach (adr)
- Average Daily Reach% (adrPer)

# Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Cross Web и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.crossweb import tasks as cwt
from mediascope_api.crossweb import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с API Cross Web
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.CrossWebTask()
cats = cwc.CrossWebCats()

## Задание

### Получим ID ресурсов
Для построения отчета необходимо получить идентификаторы ресурсов __Avito__ и __Ozon__.

Для этого воспользуемся методами поиска в медиа справочнике. Работа с медиа справочником подробно описана в ноутбуке [catalogs](catalogs.ipynb). 

Получим идентификатор ресурса **Avito**

In [ ]:
cats.get_resource(resource='Avito')

Получим идентификатор ресурса **Ozon**

In [ ]:
cats.get_resource(resource='Ozon')

Таким образом, необходимые идентификаторы следующие:

- **Avito**  resourceId = 1028
- **Ozon**  resourceId = 1094

### Расчет пересечения аудитории интернет-ресурсов

In [ ]:
%%time

# Задаем название для отображения в DataFrame
project_name = 'Duplication'

# Задаем период
# Период указывается в виде списка ('Начало', 'Конец'). Можно указать несколько периодов
date_filter = [('2022-08-01', '2022-08-31')]

# Задаем фильтр по типам пользования интернетом
usetype_filter = [1,2,3]

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Задаем фильтр по демографии, в нашем случае он не требуется
demo_filter = None

# Задаем фильтр по медиа и фильтр по пересечению, в нашем случае это ID ресурсов Avito и Ozon
mart_filter = 'crossMediaResourceId in (1028, 1094)'
duplication_mart_filter = 'crossMediaResourceId in (1028, 1094)'

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["crossMediaResourceName", "duplicationCrossMediaResourceId"]

# Указываем список статистик для расчета
statistics = ["reach", "reachPer", "adr", "adrPer"]

# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task_media_duplication('media-duplication', project_name, date_filter=date_filter, 
                             usetype_filter=usetype_filter, geo_filter=geo_filter,
                             demo_filter=demo_filter, mart_filter=mart_filter,
                             duplication_mart_filter=duplication_mart_filter,
                             slices=slices, statistics=statistics, scales=None)

# Отправляем задание на расчет и ждем выполнения
task = mtask.wait_task(mtask.send_media_duplication_task(task_json))

# Получаем результат
df_result = mtask.result2table(mtask.get_result(task), project_name = project_name)
df_result

## Сохраняем в Excel

In [ ]:
writer = pd.ExcelWriter(mtask.get_excel_filename('duplication'))
df_info = mtask.get_report_info()
df_result.to_excel(writer, 'Report', index=False)
df_info.to_excel(writer, 'Info', index=False)
writer.save()